# 🍓 Task 2: Strawberry Ripeness Classification

This notebook trains an **EfficientNet-B0** model to classify strawberries into 3 categories:
0.  **Ripe** (Green label in COCO)
1.  **Unripe** (Red label)
2.  **Half-Ripe** (Orange label)

**Workflow:**
1.  **Download Dataset** (from GitHub Releases).
2.  **Crop Extraction**: Use ground-truth bounding boxes to extract individual strawberry images.
3.  **Data Augmentation**: Apply transforms (Flip, Rotate, Color Jitter).
4.  **Training**: Fine-tune a pre-trained EfficientNet model.

## 1. Setup & Download

Standard setup block to get the dataset ready.

In [ ]:
import os
import sys
import json
import requests
import zipfile
import shutil
import glob
import inspect
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm
from pathlib import Path
import cv2
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
    import timm


# --- Robust Dataset Configuration ---
VERSION_TAG = "Dataset"
BASE_URL = f"https://github.com/SergKurchev/strawberry_synthetic_dataset/releases/download/{VERSION_TAG}"
FILES_TO_DOWNLOAD = [
    "strawberry_dataset.zip.001",
    "strawberry_dataset.zip.002",
    "strawberry_dataset.zip.003"
]
OUTPUT_ZIP = "strawberry_dataset.zip"

def reconstruct_metadata(dataset_root):
    """Reconstructs depth_metadata.json from individual files in metadata_temp/"""
    print("⚠️ 'depth_metadata.json' not found. Attempting reconstruction from 'metadata_temp/'...")
    temp_dir = dataset_root / "metadata_temp"
    if not temp_dir.exists():
        print(f"❌ metadata_temp directory not found at {temp_dir}")
        return False

    combined_metadata = {}
    json_files = list(temp_dir.glob("*_meta.json"))
    print(f"  Found {len(json_files)} metadata chunks.")
    
    for json_file in tqdm(json_files, desc="Reconstructing Metadata"):
        try:
            # Filename format: 00001_meta.json -> corresponds to 00001.png
            # We assume the content of the json is the metadata dict for that image
            img_id = json_file.name.replace("_meta.json", "")
            img_name = f"{img_id}.png"
            
            with open(json_file, 'r') as f:
                data = json.load(f)
                combined_metadata[img_name] = data
        except Exception as e:
            print(f"  Warning: Failed to read {json_file}: {e}")

    if not combined_metadata:
        print("❌ Failed to reconstruct any metadata.")
        return False

    target_path = dataset_root / "depth_metadata.json"
    print(f"💾 Saving reconstructed metadata to {target_path}...")
    with open(target_path, 'w') as f:
        json.dump(combined_metadata, f, indent=2)
        
    return True

def setup_dataset():
    # 1. Search for existing dataset
    print("🔍 Searching for existing dataset...")
    
    # Helper to validate a root candidate
    def validate_root(p):
        if (p / "depth_metadata.json").exists():
            return True
        if (p / "metadata_temp").exists():
            # Try to fix it
            return reconstruct_metadata(p)
        return False

    # Recursive search in current dir
    for root, dirs, files in os.walk(".", topdown=True):
        p = Path(root)
        # Start optimization: don't go too deep or into hidden dirs
        if ".git" in p.parts or "temp_download" in p.parts:
            continue
            
        if "images" in dirs and ("depth_metadata.json" in files or "metadata_temp" in dirs):
            if validate_root(p):
                print(f"✅ Dataset found/Fixed at: {p}")
                return p

    # Check standard paths
    search_paths = [
        Path("strawberry_dataset"),
        Path("dataset/strawberry_dataset"),
        Path("/kaggle/input/last-straw-dataset/strawberry_dataset"),
        Path("/kaggle/input/strawberry_synthetic_dataset/strawberry_dataset")
    ]
    for p in search_paths:
        if p.exists():
            if validate_root(p):
                print(f"✅ Dataset found/Fixed at: {p}")
                return p

    print("⬇️ Dataset not found. Downloading from GitHub Releases...")
    
    # 2. Prepare Download Directory
    if os.path.exists("temp_download"):
        shutil.rmtree("temp_download")
    os.makedirs("temp_download", exist_ok=True)
    
    if os.path.exists(OUTPUT_ZIP):
        os.remove(OUTPUT_ZIP)

    # 3. Download and Combine
    with open(OUTPUT_ZIP, 'wb') as outfile:
        for filename in FILES_TO_DOWNLOAD:
            file_path = Path("temp_download") / filename
            url = f"{BASE_URL}/{filename}"
            
            print(f"  Downloading {filename} from {url}...")
            r = requests.get(url, stream=True)
            if r.status_code != 200:
                raise RuntimeError(f"Download failed for {filename}: HTTP {r.status_code}")
            
            with open(file_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            file_size_mb = file_path.stat().st_size / 1024 / 1024
            print(f"  Downloaded {filename} ({file_size_mb:.2f} MB). Appending to zip...")
            
            with open(file_path, 'rb') as infile:
                shutil.copyfileobj(infile, outfile)

    # 4. Extract
    total_size_mb = os.path.getsize(OUTPUT_ZIP)/1024/1024
    print(f"📂 Extracting {OUTPUT_ZIP} ({total_size_mb:.2f} MB)...")
    
    try:
        with zipfile.ZipFile(OUTPUT_ZIP, 'r') as zip_ref:
            zip_ref.extractall(".")
            print("  Extraction complete.")
    except zipfile.BadZipFile as e:
        print(f"❌ BadZipFile Error: {e}")
        raise e
    
    shutil.rmtree("temp_download", ignore_errors=True)
    if os.path.exists(OUTPUT_ZIP):
        os.remove(OUTPUT_ZIP)

    # --- FIX: Handle potential backslash filenames on Linux ---
    print("🧹 Checking for backslash issues in filenames...")
    count = 0
    # Iterate over files in current directory to check for backslashes in names
    for filename in os.listdir("."):
        if "\\" in filename:
            # It's a file with backslashes in name, implying flattened structure
            new_path = filename.replace("\\", "/") # standardize to forward slash
            
            # Create parent dirs
            parent = os.path.dirname(new_path)
            if parent:
                os.makedirs(parent, exist_ok=True)
            
            # Move file
            try:
                shutil.move(filename, new_path)
                count += 1
            except Exception as e:
                print(f"  Failed to move {filename} -> {new_path}: {e}")
            
    if count > 0:
        print(f"✅ Fixed {count} filenames with backslashes. Directory structure restored.")
        
    # 5. Locate and Fix
    print("🔎 Locating dataset root...")
    for root, dirs, files in os.walk(".", topdown=True):
        p = Path(root)
        if "images" in dirs and ("depth_metadata.json" in files or "metadata_temp" in dirs):
            if validate_root(p):
                 print(f"✅ Dataset extracted and verified at: {p}")
                 return p
            
    return None

DATASET_PATH = setup_dataset()
if not DATASET_PATH: raise RuntimeError("Dataset setup failed: Could not locate or reconstruct metadata")
DATASET_ROOT = DATASET_PATH


## 2. Crop Extraction

We need to cut out each strawberry from the full images based on the annotations to create a classification dataset.

In [ ]:
# Load Annotations
with open(DATASET_PATH / "annotations.json", 'r') as f:
    coco = json.load(f)

CROPS_DIR = Path("strawberry_crops")
shutil.rmtree(CROPS_DIR, ignore_errors=True)
CROPS_DIR.mkdir(exist_ok=True)

# Categories we care about for classification
CLASS_MAP = {
    0: "ripe",
    1: "unripe",
    2: "half_ripe"
}

for name in CLASS_MAP.values():
    (CROPS_DIR / name).mkdir(exist_ok=True)

print("✂️ Extracting crops...")
count = 0

# Optimize: Group anns by image_id
from collections import defaultdict
img_to_anns = defaultdict(list)
for ann in coco['annotations']:
    img_to_anns[ann['image_id']].append(ann)

for img_info in tqdm(coco['images']):
    img_id = img_info['id']
    if img_id not in img_to_anns: continue
    
    # Load Image
    img_path = DATASET_PATH / "images" / img_info['file_name']
    if not img_path.exists(): continue
    
    # We read with OpenCV for faster cropping
    img = cv2.imread(str(img_path))
    if img is None: continue
    
    for ann in img_to_anns[img_id]:
        cat_id = ann['category_id']
        if cat_id not in CLASS_MAP: continue
        
        x, y, w, h = [int(v) for v in ann['bbox']]
        
        # Padding usually helps classification models
        pad = 10
        x = max(0, x - pad)
        y = max(0, y - pad)
        w = min(img.shape[1] - x, w + 2*pad)
        h = min(img.shape[0] - y, h + 2*pad)
        
        crop = img[y:y+h, x:x+w]
        if crop.size == 0: continue
        
        save_name = f"{img_info['file_name']}_{ann['id']}.jpg"
        save_path = CROPS_DIR / CLASS_MAP[cat_id] / save_name
        cv2.imwrite(str(save_path), crop)
        count += 1

print(f"✅ Extracted {count} crops.")

## 3. Training Pipeline

We use PyTorch + timm to fine-tune EfficientNet-B0.

In [ ]:
# Dataset Class
class StrawberryDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self): return len(self.file_paths)
    
    def __getitem__(self, idx):
        img = Image.open(self.file_paths[idx]).convert("RGB")
        if self.transform: img = self.transform(img)
        return img, torch.tensor(self.labels[idx], dtype=torch.long)

# Gather files
all_files = []
all_labels = []
label_map = {"ripe": 0, "unripe": 1, "half_ripe": 2}

for class_name, label_idx in label_map.items():
    files = list((CROPS_DIR / class_name).glob("*.jpg"))
    all_files.extend(files)
    all_labels.extend([label_idx] * len(files))

# Split
train_files, val_files, train_labels, val_labels = train_test_split(
    all_files, all_labels, test_size=0.2, stratify=all_labels, random_state=42
)

# Transforms
train_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Loaders
train_ds = StrawberryDataset(train_files, train_labels, train_tf)
val_ds = StrawberryDataset(val_files, val_labels, val_tf)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=3)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print(f"🚀 Training on {device} with {len(train_ds)} samples...")

for epoch in range(5):  # Short training for demo
    model.train()
    running_loss = 0.0
    for imgs, lbls in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        imgs, lbls = imgs.to(device), lbls.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, lbls)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"  Loss: {running_loss/len(train_loader):.4f}")

print("✅ Training Complete.")

## 4. Evaluation

Check accuracy and confusion matrix.

In [ ]:
model.eval()
all_preds = []
all_true = []

with torch.no_grad():
    for imgs, lbls in tqdm(val_loader, desc="Validating"):
        imgs, lbls = imgs.to(device), lbls.to(device)
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_true.extend(lbls.cpu().numpy())

print("\nClassification Report:")
print(classification_report(all_true, all_preds, target_names=label_map.keys()))